# Text Classification Model

## Load Data

## Data Exploration

## Data Preparation & Pre-processing

## Modelling

### Naive Bayes

### Logistics Regression

### Support Vector Machine

### Random Forest